In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [11]:
df = pd.read_csv('comm173_youtube.csv')

In [12]:
print(df)

                                                  link   
0    https://www.youtube.com/watch?v=3qTZnwRbFY0&pp...  \
1    https://www.youtube.com/watch?v=jBFMDZ5ugd4&pp...   
2    https://www.youtube.com/watch?v=L_2jL8vNiNM&pp...   
3    https://www.youtube.com/watch?v=gXjcwnEpILg&pp...   
4    https://www.youtube.com/watch?v=3XDcNI31T2k&pp...   
..                                                 ...   
574  https://www.youtube.com/watch?v=PBQfw7ZIhmg&pp...   
575  https://www.youtube.com/watch?v=Xt4ILWoarrA&pp...   
576  https://www.youtube.com/watch?v=IELz64nzSbc&pp...   
577  https://www.youtube.com/watch?v=Qx_3AAKmL4Y&pp...   
578  https://www.youtube.com/watch?v=pkYBVG9p8Ao&pp...   

                                                 title  description   
0    This Is Why Costco Only Charges $5 For A Rotis...          NaN  \
1       Removing Meat from a Costco Rotisserie Chicken          NaN   
2    The Big Problem Costco Shoppers Are Currently ...          NaN   
3    Don't Eat Anot

In [13]:
df[~df.link.str.contains("/shorts")]

,link,title,description,category
0,https://www.youtube.com/watch?v=3qTZnwRbFY0&pp...,This Is Why Costco Only Charges $5 For A Rotis...,NaN,costco_chicken
1,https://www.youtube.com/watch?v=jBFMDZ5ugd4&pp...,Removing Meat from a Costco Rotisserie Chicken,NaN,costco_chicken
2,https://www.youtube.com/watch?v=L_2jL8vNiNM&pp...,The Big Problem Costco Shoppers Are Currently ...,NaN,costco_chicken
3,https://www.youtube.com/watch?v=gXjcwnEpILg&pp...,Don't Eat Another Costco Rotisserie Chicken Un...,NaN,costco_chicken
4,https://www.youtube.com/watch?v=3XDcNI31T2k&pp...,10 Easy Costco Chicken Dinners | Recipes You C...,NaN,costco_chicken
...,...,...,...,...
574,https://www.youtube.com/watch?v=PBQfw7ZIhmg&pp...,ONCE A MONTH GROCERY HAUL FOR OUR NOT SO LARGE...,NaN,costco_chicken
575,https://www.youtube.com/watch?v=Xt4ILWoarrA&pp...,Let's Make a Deal,NaN,costco_chicken
576,https://www.youtube.com/watch?v=IELz64nzSbc&pp...,The Preston & Steve Show 5/09/23,NaN,costco_chicken
577,https://www.youtube.com/watch?v=Qx_3AAKmL4Y&pp...,Cooking With The Chew Crew,NaN,costco_chicken


In [14]:
df_2 = df[~df.link.str.contains("/shorts")]

In [18]:
driver = webdriver.Chrome(('/Users/katelynwong/Documents/COMM-173/chromedriver'))

/var/folders/3l/682x0z_16p5gngb4z1mctltw0000gn/T/ipykernel_98165/1243937836.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(('/Users/katelynwong/Documents/COMM-173/chromedriver'))


In [19]:
driver.maximize_window()
driver.implicitly_wait(20)

In [20]:
comments = []
comment_likes = []

In [21]:
def scroll_to_bottom(driver):
    old_position = 0
    new_position = None
    scroll_distance = 500
    
    while (new_position != old_position):
        # Get old scroll position
        old_position = driver.execute_script(("return (window.pageYOffset !== undefined) ?"
                                            " window.pageYOffset : (document.documentElement ||"
                                            " document.body.parentNode || document.body);"))
        # Sleep and scroll
        time.sleep(5)
        driver.execute_script(("var scrollingElement = (document.scrollingElement ||"
                              " document.body);scrollingElement.scrollTop = "
                              " scrollingElement.scrollHeight"))
        
        time.sleep(5)
        # Get new position
        new_position = driver.execute_script(("return (window.pageYOffset !== undefined) ?"
                                             " window.pageYOffset : (document.documentElement ||"
                                             " document.body.parentNode || documentBody);"))

In [22]:
# Loop through the rows in the dataframe
for index, row in df_2.iterrows():
    # Navigate to the YouTube video page
    driver.get(row['link'])

    # Wait for the comments to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#comments #content-text')))
    
    scroll_to_bottom(driver)
    
    time.sleep(2)

    # Get the comment and comment like elements and extract their text
    comment_elems = driver.find_elements(By.CSS_SELECTOR,'#comments #content-text')
    comment_like_elems = driver.find_elements(By.CSS_SELECTOR,'#comments #vote-count-middle')
    comment_text = [elem.text for elem in comment_elems]
    comment_like_text = [elem.text for elem in comment_like_elems]

    # Append the comment and comment like text to the corresponding lists
    for i in range(len(comment_text)):
        comments.append([row['link'], row['title'], comment_text[i]])
        comment_likes.append([row['link'], row['title'], comment_text[i], comment_like_text[i]])

KeyboardInterrupt: 

In [ ]:
driver.quit()

In [23]:
comments_df = pd.DataFrame(comments, columns=['Link', 'Title', 'Comment'])
comment_likes_df = pd.DataFrame(comment_likes, columns=['Link', 'Title', 'Comment', 'Likes'])

In [24]:
comments_df.to_csv('video_comments.csv', index=False)
comment_likes_df.to_csv('video_comment_likes.csv', index=False)